In [39]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.patches as patches

# Face Detectors
import cv2
from mtcnn import MTCNN

import torch
import torchvision.ops.boxes as bop

import json
import os
import matplotlib.pyplot as plt
import random
import seaborn as sns
from keras.models import Sequential
from keras import optimizers
from keras import backend as K
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
annotations = "./annotations"
images = "./images"
df = pd.read_csv("train.csv")
df_test = pd.read_csv("submission.csv")

sample data from training set.

In [40]:
from helper import getJSON


files = []
for i in os.listdir(annotations):
    files.append(getJSON(os.path.join(annotations,i)))
files[0]

{'FileName': '1801.jpg',
 'NumOfAnno': 1,
 'Annotations': [{'isProtected': False,
   'ID': 924868908868875136,
   'BoundingBox': [451, 186, 895, 697],
   'classname': 'face_no_mask',
   'Confidence': 1,
   'Attributes': {}}]}

In [41]:

df.head()

,name,x1,x2,y1,y2,classname
0,2756.png,69,126,294,392,face_with_mask
1,2756.png,505,10,723,283,face_with_mask
2,2756.png,75,252,264,390,mask_colorful
3,2756.png,521,136,711,277,mask_colorful
4,6098.jpg,360,85,728,653,face_no_mask


use mask label and non_mask label to extract bounding box data from json files.

store faces from any particular image in the train list along with its label for the training process.

In [42]:
train = []
img_size = 124
mask = ['face_with_mask']
non_mask = ["face_no_mask"]
labels={'mask':0,'without mask':1}
for i in df["name"].unique():
    f = i+".json"
    for j in getJSON(os.path.join(annotations,f)).get("Annotations"):
        if j["classname"] in mask:
            x,y,w,h = j["BoundingBox"]
            img = cv2.imread(os.path.join(images,i),1)
            img = img[y:h,x:w]
            img = cv2.resize(img,(img_size,img_size))
            train.append([img,labels["mask"]])
        if j["classname"] in non_mask:
            x,y,w,h = j["BoundingBox"]
            img = cv2.imread(os.path.join(images,i),1)
            img = img[y:h,x:w]
            img = cv2.resize(img,(img_size,img_size))    
            train.append([img,labels["without mask"]])
random.shuffle(train)  
len(train)

5749

In [43]:
train[:]

[[array([[[107, 111, 107],
          [123, 127, 123],
          [116, 120, 117],
          ...,
          [ 92,  88,  74],
          [ 85,  81,  65],
          [ 81,  76,  61]],
  
         [[107, 111, 108],
          [124, 128, 124],
          [115, 120, 116],
          ...,
          [ 94,  87,  73],
          [ 89,  82,  67],
          [ 89,  80,  64]],
  
         [[108, 113, 109],
          [124, 128, 124],
          [115, 120, 116],
          ...,
          [109, 100,  82],
          [105,  93,  77],
          [105,  91,  74]],
  
         ...,
  
         [[ 72,  64,  53],
          [ 77,  67,  56],
          [ 78,  69,  58],
          ...,
          [216, 209, 204],
          [216, 207, 204],
          [215, 207, 205]],
  
         [[ 82,  75,  63],
          [ 76,  67,  55],
          [ 78,  69,  58],
          ...,
          [216, 209, 204],
          [217, 207, 204],
          [216, 207, 206]],
  
         [[ 84,  77,  65],
          [ 75,  67,  55],
          [ 80,  71,  61

prepare test data and split data and labels.

In [44]:
X = []
Y = []
for features,label in train:
    X.append(features)
    Y.append(label)
    
print(X[0].shape)
print(Y[0])
print(Y[1])

(124, 124, 3)
0
0


In [45]:
X = np.array(X)/255.0
X = X.reshape(-1,124,124,3)

Y = np.array(Y)
print(np.unique(Y))
Y.shape

[0 1]


(5749,)

Model training


In [46]:
model = Sequential()

model.add(Conv2D(32, (3, 3), padding = "same", activation='relu', input_shape=(124,124,3)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
 
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

In [47]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 124, 124, 32)      896       
                                                                 
 conv2d_4 (Conv2D)           (None, 122, 122, 64)      18496     
                                                                 
 conv2d_5 (Conv2D)           (None, 120, 120, 128)     73856     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 60, 60, 128)       0         
 g2D)                                                            
                                                                 
 dropout_3 (Dropout)         (None, 60, 60, 128)       0         
                                                                 
 flatten_1 (Flatten)         (None, 460800)            0         
                                                      

compile the model:

In [48]:
model.compile(loss='binary_crossentropy', optimizer='adam' ,metrics=['accuracy'])

split training data set to traqin and validation:

In [49]:
xtrain,xval,ytrain,yval=train_test_split(X, Y,train_size=0.8,random_state=0)

print(type(X))
print(type(Y))


<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [50]:
print(xtrain.shape)
print(ytrain.shape)


(4599, 124, 124, 3)
(4599,)


generate tensor make data using ImageDataGenerator:

In [51]:
tensordata = ImageDataGenerator(
        featurewise_center=False,  
        samplewise_center=False,  
        featurewise_std_normalization=False,  
        samplewise_std_normalization=False,  
        zca_whitening=False,    
        rotation_range=15,    
        width_shift_range=0.1,
        height_shift_range=0.1,  
        horizontal_flip=True,  
        vertical_flip=False)
tensordata.fit(xtrain)

In [1]:
# from codegreen.decorators import init_experiment, time_shift, upload_cc_report
# from codecarbon import track_emissions
# from codegreen.queries import get_location_prediction, get_data
import numpy as np
from datetime import datetime

ModuleNotFoundError: No module named 'codegreen'

 fit our model:

In [52]:
# @init_experiment(estimated_runtime_hours=1,estimated_runtime_minutes=30,percent_renewable=10,allowed_delay_hours=24,area_code="ES-9",log_request=True,experiment_name="my_experiment",codecarbon_logfile="experiment.log",nextflow_logfile="nextflow.log",overwrite=False)
# @time_shift("my_experiment")
# @upload_cc_report("my_experiment")
# @track_emissions(output_file='experiment.log')

history = model.fit(
    tensordata.flow(xtrain, ytrain, batch_size=32),
    steps_per_epoch=len(xtrain) // 32,
    epochs=50,
    verbose=1,
    validation_data=(xval, yval)
)


Epoch 1/50
143/143 [==============================] - 323s 2s/step - loss: 0.4418 - accuracy: 0.8073 - val_loss: 0.3277 - val_accuracy: 0.8896
Epoch 2/50
143/143 [==============================] - 352s 2s/step - loss: 0.3074 - accuracy: 0.8664 - val_loss: 0.2279 - val_accuracy: 0.9061
Epoch 3/50
143/143 [==============================] - 320s 2s/step - loss: 0.2944 - accuracy: 0.8820 - val_loss: 0.2629 - val_accuracy: 0.8878
Epoch 4/50
143/143 [==============================] - 352s 2s/step - loss: 0.2884 - accuracy: 0.8822 - val_loss: 0.2109 - val_accuracy: 0.9183
Epoch 5/50
143/143 [==============================] - 342s 2s/step - loss: 0.2923 - accuracy: 0.8837 - val_loss: 0.2247 - val_accuracy: 0.9148
Epoch 6/50
143/143 [==============================] - 347s 2s/step - loss: 0.2752 - accuracy: 0.8872 - val_loss: 0.2613 - val_accuracy: 0.9043
Epoch 7/50
143/143 [==============================] - 381s 3s/step - loss: 0.2630 - accuracy: 0.8988 - val_loss: 0.2069 - val_accuracy: 0.9157

KeyboardInterrupt: 